# Ejercicio validación

Para el siguiente ejemplo, se solicita importar los datos del dataset `housing_data.csv` y realizar el proceso de validación utilizando los siguientes mecanismos 

- Validación Hold-out
- Validación Cruzada

>Note: No realizar predicciones, solo las validaciones. Recordar hacer las validaciones internas. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')

## Importar datos

In [ ]:
df = pd.read_csv('student-mat.csv', sep=';')
df

In [ ]:
df = df[['age', 
           'Medu', 
           'Fedu', 
           'traveltime', 
           'studytime', 
           'failures',
           'famrel',
           'freetime',
           'goout',
           'Dalc',
           'Walc',
           'health',
           'absences',
           'G1',
           'G2',
           'G3']]

df.info()

In [ ]:
df_train=df.drop(labels='G3',axis=1)
df_label=df['G3']

## Validación Hold-out

In [ ]:
# Test: hold-out split 80-20%. # Partición externa
X_training, X_test, y_training, y_test = train_test_split(df.drop(labels='G3',axis=1), 
                                                          df.G3, 
                                                          test_size=0.2, 
                                                          random_state=42) 

valores_test, ocur_test = np.unique(y_test, return_counts=True)

print('Test: ', 'clases:', valores_test)
print('\n')
print('ocurrencias: ', ocur_test)

In [ ]:
# Estandarizar las características de entrenamiento y de test
standardizer = StandardScaler()
X_training = standardizer.fit_transform(X_training)
X_test = standardizer.transform(X_test)

In [ ]:
# Validación: hold-out split 80-20%. # Partición interna
X_train, X_val, y_train, y_val = train_test_split(X_training, 
                                                  y_training, 
                                                  test_size=0.2, 
                                                  random_state=42)

valores_train, ocur_train = np.unique(y_train, return_counts=True)
print('Entrenamiento: ', ' clases:', valores_train)
print('\n')
print('ocurrencias:', ocur_train)
print('\n')


valores_val, ocur_val = np.unique(y_val, return_counts=True)

print('Test: ', 'clases:', valores_val)
print('\n')
print('ocurrencias: ', ocur_val)

In [ ]:
# Construcción del objeto que contiene el algoritmo de aprendizaje.
# Utilizamos un algoritmo de regresión lineal
lm = LinearRegression()

In [ ]:
lm.fit(X_train, y_train) # Entrenamos modelo

val_accuracy = lm.score(X_val, y_val) # Evaluamos modelo en validación
print('Exactitud en validación ', np.round(val_accuracy*100, 2), '%')

test_accuracy = lm.score(X_test, y_test) # Evaluamos modelo en test
print('Exactitud en test ', np.round(test_accuracy*100, 2), '%')

## Validación Cruzada

In [ ]:
# Test: hold-out split 80-20%. PARTICIÓN EXTERNA
X_training, X_test, y_training, y_test = train_test_split(df.drop(labels='G3',axis=1), 
                                                          df.G3, 
                                                          test_size=0.2, 
                                                          random_state=42) 

valores_test, ocur_test = np.unique(y_test, return_counts=True)
print('Test: ', 'clases:', valores_test, ' ocurrencias: ', ocur_test)

valores_train, ocur_train = np.unique(y_train, return_counts=True)
print('Entrenamiento: ', ' clases:', valores_train, '  ocurrencias:', ocur_train)

In [ ]:
# Construcción del objeto que contiene el algoritmo de aprendizaje.
# Utilizamos un algoritmo de regresión lineal
lm = LinearRegression()

In [ ]:
# Validación, entrenamiento y evaluación del algoritmo de aprendizaje.
# en "cv = KFold(n_splits=5)" estamos haciendo un cross-validation INTERNO!

"""
En results se hace la validación cruzada
"""

results = cross_val_score(lm, 
                          X_train, 
                          y_train, 
                          cv = KFold(n_splits=5, shuffle = True, random_state = 42))

print("Resultados por bolsa: ", results)
print("Accuracy (media +/- desv.): %0.4f +/- %0.4f" % (results.mean(), results.std()))

In [ ]:
# Una vez entrenado y validado el modelo para seleccionar los mejores hyperparameters, 
# utilizamos todos los datos de "train" y "val" para entrenar el modelo definitivo

lm = lm.fit(X_train, y_train) # Entrenamiento
test_results = lm.score(X_test, y_test) # Evaluación en test
print('Exactitud en test: ', test_results*100, '%')
print('\n')
# También podemos extraer las predicciones, en lugar de directamente la accuracy
y_pred = lm.predict(X_test)
y=pd.Series(y_pred)
print('Predicciones:\n', y)
print('\n')
print('Etiquetas reales:\n', y_test)